# Flan T5 Base Version Evaluation

#### Imports

In [33]:
from enum import Enum
import math
import os
import re
from copy import deepcopy
import random
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pickle
from scipy.optimize import linear_sum_assignment
import unittest
import torch
import evaluate
import anytree
from anytree.importer import DictImporter
from torch.utils.data import DataLoader

In [2]:
K = 6
MAX_LAYERS = 8
MAX_TOKENS = 392
EMBEDDING_SIZE = 768

DATA_PATH = './dataset/'
SET_NAMES = ['train', 'validation', 'test']
MODEL = 'google/flan-t5-large'
MODEL_PATH = f'models/{MODEL.split("/")[-1]}-MathQA'
DEVICE = 'cuda:0'
NUM_MASK = '<num>'
WORKING_DIR = 'TEMP/'

OBJ_DIR = 'pickle/'


class Op(Enum):
    ADD = '+'
    SUB = '-'
    MULT = '*'
    DIV = '/'
    POW = '^'
    
class Const(Enum):
    CONST_NEG_1 = 'const_neg_1' # I added this
    CONST_0_25 = 'const_0_25'
    CONST_0_2778 = 'const_0_2778'
    CONST_0_33 = 'const_0_33'
    CONST_0_3937 = 'const_0_3937'
    CONST_1 = 'const_1'
    CONST_1_6 = 'const_1_6'
    CONST_2 = 'const_2'
    CONST_3 = 'const_3'
    CONST_PI = 'const_pi'
    CONST_3_6 = 'const_3_6'
    CONST_4 = 'const_4'
    CONST_5 = 'const_5'
    CONST_6 = 'const_6'
    CONST_10 = 'const_10'
    CONST_12 = 'const_12'
    CONST_26 = 'const_26'
    CONST_52 = 'const_52'
    CONST_60 = 'const_60'
    CONST_100 = 'const_100'
    CONST_180 = 'const_180'
    CONST_360 = 'const_360'
    CONST_1000 = 'const_1000'
    CONST_3600 = 'const_3600'

values = [-1, 0.25, 0.2778, 0.33, 0.3937, 1, 1.6, 2, 3, 3.1416, 3.6, 4, 5, 6, 10, 12, 26, 52, 60, 100, 180, 360, 1000, 3600]
const2val = {k:v for k,v in zip(Const._value2member_map_.keys(), values)}    

op2id = {k:v for k,v in zip(Op._value2member_map_.keys(), range(len(Op._value2member_map_)))}
op2id['None'] = 5
id2op = np.array(list(op2id.keys()))
const2id = {k:v for k,v in zip(Const._value2member_map_.keys(), range(len(Const._value2member_map_)))}
id2const = np.array(list(const2id.keys()))

torch.set_printoptions(sci_mode=False)

class Util():
    def load_obj(self, path):
        with open(path, 'rb') as f:
            o = pickle.load(f)
        return o
    
    def save_obj(self, path, o):
        with open(path, 'wb') as f:
            pickle.dump(o, f)
            
    def load_data(self):
        return {name:pd.read_csv(f'{DATA_PATH}{name}.csv') for name in SET_NAMES}
    
    def set_seed(self, seed):
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
util = Util()

## Constructing the Prompts

In [3]:
def flatten(arr):
    idx = np.concatenate([[i]*len(x) for i,x in enumerate(arr)])
    flattened = np.concatenate(arr) 
    return idx, flattened

def get_nums_and_mask(data):
    nums = {name:[] for name in SET_NAMES}
    problems = {name:[] for name in SET_NAMES}
    num_idx = {name:[] for name in SET_NAMES}
    for name in SET_NAMES:
        for i,problem in enumerate(data[name].problem):
            num = re.compile('([+-]?((\d+(\.\d*)?)|(\.\d+)))') # normal num
            big = re.compile(r'(-?\d{1,3}(,\d{3})+(\.\d*)?)') # num with comma

            big_results = re.finditer(big, problem)
            problem = re.sub(big, NUM_MASK, problem)        
            num_results = re.finditer(num, problem)
            problem = re.sub(num, NUM_MASK, problem)

            # Getting the combined numbers in order of occurence
            combined = [x for x in num_results]
            combined.extend([x for x in big_results])
            combined = sorted(combined, key=lambda x: x.start(0))

            combined = [float(x.group(0).replace(',','')) for x in combined]

            nums[name].append(np.array(combined))
            problems[name].append(problem)
        num_idx[name], nums[name] = flatten(np.array(nums[name], dtype=object))
        problems[name] = np.array(problems[name])
    return {name:{'idx':torch.tensor(num_idx[name]), 'literals':nums[name]} for name in SET_NAMES}, problems

In [4]:
data = util.load_data()

In [5]:
op = util.load_obj(f'{OBJ_DIR}ops.pickle')
const = util.load_obj(f'{OBJ_DIR}constants.pickle')
subexp = util.load_obj(f'{OBJ_DIR}subexp.pickle')
nums,_ = get_nums_and_mask(data)

In [6]:
# text preprocessing
def determine_if_int(num):
    if float(num).is_integer():
        return int(float(num))
    else:
        return float(num)

def str_numpy(arr, t=None):
    if t == 'num':
        convert = lambda x: str(determine_if_int(x))
    elif t == 'eq':
        split = lambda x: x.split()
        convert_split = lambda x: f'{x[1]} {process_num(x[0])} {process_num(x[2])}'
        convert = lambda x: convert_split(split(x))        
    else:
        convert = lambda x: str(x)
        
    output = '{'
    if len(arr) > 0:
        output += f'{convert(arr[0])}'
        for x in arr[1:]:
            output += f', {convert(x)}'
    output += '}'
    return output   

def process_num(num):
    if num in const2val:
        return str(determine_if_int(const2val[num]))
    else:
        return str(determine_if_int(num))
    
# label preprocessing
importer =  DictImporter()
def process_item(item):
    if item in id2op:
        return str(item)
    elif item in const2val:
        return str(determine_if_int(const2val[item]))
    else:
        return str(determine_if_int(item))

def convert_to_preorder(name):
    labels = []
    for tree in data[name]['tree']:
        root = importer.import_(eval(tree))
        output = ''
        for node in anytree.PreOrderIter(root):
            output += process_item(node.name) + ' '
        labels.append(output[0:-1])
    return labels
     
# main function
def prompt_engineering(name):
    eq, eq_idx = subexp[name]
    eq = np.array(eq)
    eq_idx = np.array(eq_idx)
    idx = nums[name]['idx']
    literals = nums[name]['literals']
    engineered = []
    
    for i in range(len(data[name])):
        ops = str_numpy(id2op[0:-1][op['pred'][name][i].astype(bool)])
        constants = id2const[const['pred'][name][i].astype(bool)]
        constants = str_numpy([const2val[x] for x in constants], t='num')
        numbers = str_numpy(literals[idx==i], t='num')
        equations = str_numpy(eq[eq_idx==i], t='eq')
        problem = data[name]['problem'][i]
        prompt = f'Find the mathematical formula given: numbers: {numbers}, constants: {constants}, operations: {ops}, and potential subexpressions: {equations} for problem: "{problem}"'
        engineered.append(prompt)
        
    return engineered

#labels = {name:data[name]['formula_no_const'].str.replace(r'(\d)\.0(\s|\))', r'\1\2', regex=True).str.replace(' ','') for name in SET_NAMES}
engineered = {name:Dataset.from_dict({'text':prompt_engineering(name), 'label':convert_to_preorder(name)}) for name in SET_NAMES}
engineered = DatasetDict(engineered)

In [7]:
engineered['train'][0]

{'text': 'Find the mathematical formula given: numbers: {3, 10, 36}, constants: {1, 2, 100}, operations: {*, /}, and potential subexpressions: {* 3 10, * 36 100} for problem: "the banker \' s gain of a certain sum due 3 years hence at 10 % per annum is rs . 36 . what is the present worth ?"',
 'label': '/ * 100 / * 36 100 * 3 10 * 3 10'}

## Tokenization

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)
data_collator = DataCollatorForSeq2Seq(tokenizer)
max_source = 512
max_target = 186 # The amount needed to emcompass all equations

In [9]:
def tokenization(batch):
    tokenized = tokenizer(text = batch['text'], max_length = max_source, truncation = True)
    labels = tokenizer(text_target = batch['label'], max_length = max_source, truncation = True)
    tokenized['labels'] = labels['input_ids']   
    return tokenized

tokenized = engineered.map(tokenization, batched=True, remove_columns=['text','label'])

Map:   0%|          | 0/18215 [00:00<?, ? examples/s]

Map:   0%|          | 0/2710 [00:00<?, ? examples/s]

Map:   0%|          | 0/1798 [00:00<?, ? examples/s]

## Evaluation

In [11]:
batch_size = 2
grad_acc = 4
args = Seq2SeqTrainingArguments(
    WORKING_DIR,
    evaluation_strategy='steps',
    eval_steps=1000,
    logging_strategy="steps",
    logging_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=5e-5, # learning rates around this worked well for other models trained with this dataset
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=grad_acc,
    weight_decay=.01, # to maintain consistency with pytorch AdamW optimizer
    save_total_limit=3, # ensures dont run out of disk space
    num_train_epochs=5,
    predict_with_generate=True, # allows the use of rouge and bleu metrics
    load_best_model_at_end=True,
    #optim = 'adamw_bnb_8it', # Have to use a quantized optimizer to make it possible to fit within memory
    metric_for_best_model="exact_match",
)

In [12]:
rouge_metric = evaluate.load('rouge')
bleu_metric = evaluate.load('bleu')
def compute_metrics(p):
    predictions = p[0]
    labels = p[1]
    pred_decode = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # the data collator will pad labels with -100 (to signal that they should not be used in loss calculation)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    label_decode = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # rouge (recall)
    scores = rouge_metric.compute(predictions=pred_decode, references=label_decode, use_stemmer=False)
    
    # exact match
    exact = (np.array(pred_decode)==np.array(label_decode)).sum()/len(pred_decode)
    scores['exact_match'] = exact
    
    # getting scores
    return scores

Setting a seed for consistent results upon rerun

In [13]:
util.set_seed(3)

In [14]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.evaluate(tokenized['train'])

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.22416742146015167,
 'eval_rouge1': 0.8929189520900334,
 'eval_rouge2': 0.8182982474780891,
 'eval_rougeL': 0.8698482791628894,
 'eval_rougeLsum': 0.8697759898913384,
 'eval_exact_match': 0.5994510019214933,
 'eval_runtime': 2995.3355,
 'eval_samples_per_second': 6.081,
 'eval_steps_per_second': 0.76}

In [16]:
trainer.evaluate(tokenized['validation'])

{'eval_loss': 0.45524704456329346,
 'eval_rouge1': 0.851220752772899,
 'eval_rouge2': 0.7371863917061456,
 'eval_rougeL': 0.8182458846701604,
 'eval_rougeLsum': 0.8183690287669403,
 'eval_exact_match': 0.5243542435424354,
 'eval_runtime': 450.4726,
 'eval_samples_per_second': 6.016,
 'eval_steps_per_second': 0.753}

In [17]:
trainer.evaluate(tokenized['test'])

{'eval_loss': 0.4675537645816803,
 'eval_rouge1': 0.8562289607094349,
 'eval_rouge2': 0.7378416279901248,
 'eval_rougeL': 0.8214545453461344,
 'eval_rougeLsum': 0.8214815098924417,
 'eval_exact_match': 0.5305895439377085,
 'eval_runtime': 310.6798,
 'eval_samples_per_second': 5.787,
 'eval_steps_per_second': 0.724}

In [41]:
generated = []
for x in DataLoader(tokenized['validation'], batch_size=8, collate_fn=data_collator):
    ids = model.generate(x['input_ids'].cuda())
    pred = tokenizer.batch_decode(ids, skip_special_tokens=True)
    generated.append(pred)

D:\anaconda\envs\torch\Lib\site-packages\transformers\generation\utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


T5 does not recognize '^' as a token and replaces it with \<unk\>

In [55]:
[x for batch in generated for x in batch][0:20]

[' 5 4',
 '/ * * 2 3 10 * * 2 3 10',
 '/ 72 * 6 210',
 '- / * 1 1000 / 48 12 12',
 '/ * 650 - 100 + + 44 28 10 100',
 '/ 1 - / 1 3 - / 1 2 / 1 4',
 '/ - 160 90 160',
 '/ 800 - 1 / 1 3',
 '- 10 1',
 '/  16 / 1 2  25 / 1 2',
 '*  / * 1348.32 100 2 + 1 / 1 2 100',
 '/ - 282 * 10 6.2 40',
 '/ 135 - 1 + / 1 2 * / 1 2 /',
 '/ 600 - * 54 0.2778 * 36 0.2778',
 '* 2 * 3.1416 7',
 '/ / * 32 3.8 1.6 / / * 32 3.8 ',
 '+ / - * 7 2 * 7 2 3 - * 7 2 * 7',
 '* 2 / * 20 24 + 24 21',
 '- * 1200  + 1 / 20 100 6 1200',
 '/ * * * * * * * * * * * * * * * * *']

In [56]:
with_special = []
for x in DataLoader(tokenized['validation'], batch_size=8, collate_fn=data_collator):
    ids = model.generate(x['input_ids'].cuda())
    pred = tokenizer.batch_decode(ids, skip_special_tokens=False)
    with_special.append(pred)

In [66]:
real_generated=[x.replace('<pad>','').replace('<unk>','^').replace('</s>','').strip() for batch in with_special for x in batch]

Replacing \<unk\> with '^' and seeing if it affects the accuracy. (it doesnt since the \<unk\> token is ignored for both true and predicted labels in the metrics function)

In [70]:
(np.array(real_generated)==np.array(engineered['validation']['label'])).sum()/len(real_generated)

0.5243542435424354

In [73]:
real_generated[0:20]

['^ 5 4',
 '/ * * 2 3 10 * * 2 3 10',
 '/ 72 * 6 210',
 '- / * 1 1000 / 48 12 12',
 '/ * 650 - 100 + + 44 28 10 100',
 '/ 1 - / 1 3 - / 1 2 / 1 4',
 '/ - 160 90 160',
 '/ 800 - 1 / 1 3',
 '- 10 1',
 '/ ^ 16 / 1 2 ^ 25 / 1 2',
 '* ^ / * 1348.32 100 2 + 1 / 1 2 100',
 '/ - 282 * 10 6.2 40',
 '/ 135 - 1 + / 1 2 * / 1 2 /',
 '/ 600 - * 54 0.2778 * 36 0.2778',
 '* 2 * 3.1416 7',
 '/ / * 32 3.8 1.6 / / * 32 3.8',
 '+ / - * 7 2 * 7 2 3 - * 7 2 * 7',
 '* 2 / * 20 24 + 24 21',
 '- * 1200 ^ + 1 / 20 100 6 1200',
 '/ * * * * * * * * * * * * * * * * *']

In [74]:
engineered['validation']['label'][0:20]

['^ 5 4',
 '- 1 ^ / 1 2 3',
 '/ 72 * 6 210',
 '- / * 1 1000 / 48 12 12',
 '/ * 650 - 100 + + 44 28 10 100',
 '/ 1 - / 1 3 - / 1 2 / 1 4',
 '/ - 160 90 160',
 '/ / 800 - 1 / 1 3 - 1 / 1 3',
 '* * -1 + 2 3 - 10 + 2 3',
 '/ ^ 16 / 1 2 ^ 25 / 1 2',
 '- ^ / * 1348.32 100 / 1200 100 / 1 2 ^ / * 1200 100 / 1200 100 / 1 2',
 '/ - 282 * 10 6.2 40',
 '/ 135 - 1 + / 1 2 * / 1 2 / 1 4',
 '/ 600 - * 54 0.2778 * 36 0.2778',
 '* 2 * 3.1416 7',
 '/ * / * 32 3.8 1.6 3.9 5.7',
 '+ / - * * 7 2 3 * 7 2 2 - * * 7 2 3 * 7 2',
 '* * / * 20 24 + 24 21 21 2',
 '- * 1200 ^ + 1 / 20 100 6 1200',
 '+ + * 8 100 * 1 10 9']

Correct example

In [80]:
engineered['validation']['text'][9]

'Find the mathematical formula given: numbers: {25, 16}, constants: {1, 2, 3, 4, 100}, operations: {-, /, ^}, and potential subexpressions: {/ 1 2} for problem: "two trains , one from howrah to patna and the other from patna to howrah , start simultaneously . after they meet , the trains reach their destinations after 25 hours and 16 hours respectively . the ratio of their speeds is ?"'

In [83]:
print(f'True: {real_generated[9]}')
print(f"Pred: {engineered['validation']['label'][9]}")

True: / ^ 16 / 1 2 ^ 25 / 1 2
Pred: / ^ 16 / 1 2 ^ 25 / 1 2


In [90]:
engineered['validation']['text'][14]

'Find the mathematical formula given: numbers: {7}, constants: {0.2778, 1, 2, 3, 3.1416, 3.6, 4, 10, 60, 100, 1000, 3600}, operations: {+, *, ^}, and potential subexpressions: {* 3.1416 7} for problem: "calculate the circumference of a circular field whose radius is 7 centimeters ."'

In [91]:
print(f'True: {real_generated[14]}')
print(f"Pred: {engineered['validation']['label'][14]}")

True: * 2 * 3.1416 7
Pred: * 2 * 3.1416 7


Incorrect example

In [85]:
engineered['validation']['text'][17]

'Find the mathematical formula given: numbers: {20, 21, 24}, constants: {1, 2, 3, 10}, operations: {+, *, /}, and potential subexpressions: {* 20 24, + 21 24, + 24 21} for problem: "a woman complete a journey in 20 hours . she travels first half of the journey at the rate of 21 km / hr and second half at the rate of 24 km / hr . find the total journey in km ."'

In [87]:
print(f'True: {real_generated[17]}')
print(f"Pred: {engineered['validation']['label'][17]}")

True: * 2 / * 20 24 + 24 21
Pred: * * / * 20 24 + 24 21 21 2


In [88]:
engineered['validation']['text'][19]

'Find the mathematical formula given: numbers: {1, 0, 7, 9, 2, 0, 3, 2, 3, 1, 0, 8, 4, 9, 2, 6, 5, 6, 7, 8}, constants: {1, 2, 3, 4, 10}, operations: {+, -, *, /, ^}, and potential subexpressions: {} for problem: "you need to unlock a secret code using following clues , can you ? here you have the clues : clue - 1 : 0 7 9 ( one of the numbers is correct and is placed in its correct position ) clue - 2 : 0 3 2 ( nothing is correct ) clue - 3 : 1 0 8 ( two numbers are correct but not placed at its correct position . ) clue - 4 : 9 2 6 ( one number is correct but not placed at its correct position . ) clue - 5 : 6 7 8 ( one number is correct but not placed at its correct position . )"'

In [89]:
print(f'True: {real_generated[19]}')
print(f"Pred: {engineered['validation']['label'][19]}")

True: / * * * * * * * * * * * * * * * * *
Pred: + + * 8 100 * 1 10 9
